In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [38]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(64 * 8 * 8, 128)
    self.fc2 = nn.Linear(128, 10)
    self.pool = nn.MaxPool2d(2, 2)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.5)


  def forward(self, x):
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))
    x = x.view(-1, 64 * 8 * 8)
    x = self.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.fc2(x)
    return x

In [39]:
def train_model(model, train_loader, criterion, optimizer, device):
  model.train()
  running_loss = 0.0
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  return running_loss / len(train_loader)

In [40]:
def evaluate_model(model, test_loader, device):
  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
    for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    return 100 * correct / total

In [ ]:
def load_data(data_augmentation=False):
  transforms_original = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])

  transforms_augmented = transforms.Compose([
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(10),
      transforms.RandomCrop(32, padding=4),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])

  if data_augmentation:
    original_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_original)
    augmented_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_augmented)
    train_dataset = torch.utils.data.ConcatDataset([original_dataset, augmented_dataset])

  else:
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_original)

  test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_original)

  train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
  test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)

  return train_loader, test_loader

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 25

In [46]:
train_loader, test_loader = load_data(data_augmentation=False)
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Without Augmentation:")
for epoch in range(epochs):
  loss = train_model(model, train_loader, criterion, optimizer, device)
  print(f"Epoch: {epoch}: Loss = {loss}")

accuracy = evaluate_model(model, test_loader, device)
print(f"Accuracy: {accuracy}")

Files already downloaded and verified
Files already downloaded and verified
Without Augmentation:
Epoch: 0: Loss = 1.5134301941717982
Epoch: 1: Loss = 1.1812388984596027
Epoch: 2: Loss = 1.0412691630365905
Epoch: 3: Loss = 0.9549658823653561
Epoch: 4: Loss = 0.8901301381533103
Epoch: 5: Loss = 0.8299801355356451
Epoch: 6: Loss = 0.7826952287912978
Epoch: 7: Loss = 0.7351405612952874
Epoch: 8: Loss = 0.6978789530980313
Epoch: 9: Loss = 0.6616982579078821
Epoch: 10: Loss = 0.6284299698632086
Epoch: 11: Loss = 0.6037821820782273
Epoch: 12: Loss = 0.5722921944182852
Epoch: 13: Loss = 0.5439859218609607
Epoch: 14: Loss = 0.5253340109534885
Epoch: 15: Loss = 0.5031218079235548
Epoch: 16: Loss = 0.4748791927457466
Epoch: 17: Loss = 0.46669548115385767
Epoch: 18: Loss = 0.4442000718372862
Epoch: 19: Loss = 0.43050795477217113
Epoch: 20: Loss = 0.4155505223728507
Epoch: 21: Loss = 0.4045984422992867
Epoch: 22: Loss = 0.39181108147744326
Epoch: 23: Loss = 0.38453829947792356
Epoch: 24: Loss = 0.

In [49]:
train_loader, test_loader = load_data(data_augmentation=True)
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("With Augmentation:")
for epoch in range(epochs):
  loss = train_model(model, train_loader, criterion, optimizer, device)
  print(f"Epoch: {epoch}: Loss = {loss}")

accuracy = evaluate_model(model, test_loader, device)
print(f"Accuracy: {accuracy}")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
With Augmentation:


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch: 0: Loss = 1.5169430177942425
Epoch: 1: Loss = 1.21556671719786
Epoch: 2: Loss = 1.1099427809184435
Epoch: 3: Loss = 1.0445241218031178
Epoch: 4: Loss = 1.001149382945138
Epoch: 5: Loss = 0.9669230734775711
Epoch: 6: Loss = 0.9382103514152693
Epoch: 7: Loss = 0.9176652699575467
Epoch: 8: Loss = 0.8942841886330017
Epoch: 9: Loss = 0.8797109567501266
Epoch: 10: Loss = 0.8659038106295366
Epoch: 11: Loss = 0.8447634706456006
Epoch: 12: Loss = 0.8354767283719088
Epoch: 13: Loss = 0.823828529812019
Epoch: 14: Loss = 0.8120935631538154
Epoch: 15: Loss = 0.8035225557426726
Epoch: 16: Loss = 0.796426887837878
Epoch: 17: Loss = 0.788956889340455
Epoch: 18: Loss = 0.7828546862188853
Epoch: 19: Loss = 0.768519638424414
Epoch: 20: Loss = 0.7625921787318708
Epoch: 21: Loss = 0.7572009007250431
Epoch: 22: Loss = 0.7481122898773284
Epoch: 23: Loss = 0.7415865260061361
Epoch: 24: Loss = 0.740812817255961
Accuracy: 76.55
